In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

label = train_data["label"].values

In [2]:
from keras.preprocessing.text import Tokenizer

MAX_COUNT = 10000

word_tokenizer = Tokenizer(MAX_COUNT)
word_tokenizer.fit_on_texts(question_data["words"])

word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

char_tokenizer = Tokenizer(MAX_COUNT)
char_tokenizer.fit_on_texts(question_data["chars"])

char_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, char_embedding_data.shape[1]), dtype=np.float64),
        char_embedding_data.loc[list(char_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

word_embedding_data.shape, char_embedding_data.shape

Using TensorFlow backend.


((10001, 300), (3049, 300))

In [3]:
from keras.preprocessing.sequence import pad_sequences

SEQ_LEN = 25

def gen_word_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=SEQ_LEN, padding="pre",truncating="pre")
    
def gen_char_data(data):
    seq_char1 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["chars"])
    seq_char2 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["chars"])
    return pad_sequences(seq_char1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_char2, maxlen=SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_word_data(train_data)
char1, char2 = gen_char_data(train_data)
test_word1, test_word2 = gen_word_data(test_data)
test_char1, test_char2 = gen_char_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape, char1.shape, char2.shape, test_char1.shape, test_char2.shape

((254386, 25),
 (254386, 25),
 (172956, 25),
 (172956, 25),
 (254386, 25),
 (254386, 25),
 (172956, 25),
 (172956, 25))

In [4]:
from keras.models import Model
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, Nadam, SGD
from keras.layers import LSTM, Bidirectional, TimeDistributed, CuDNNLSTM
from keras.layers import Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D, MaxPool1D
from keras.layers import Input, Embedding, Dropout, BatchNormalization, Dense, Flatten, Lambda, K, Activation

In [5]:
# general
BATCH_SIZE = 512
NUM_EPOCHES = 30
DROP_RATE = 0.3
PATIENCE = 8
# cnn
CONV_LEN_1 = 128
CONV_LEN_2 = 128
CONV_LEN_3 = 128
CONV_LEN_4 = 128
CONV_LEN_5 = 128
CONV_LEN_6 = 128
CONV_LEN = CONV_LEN_1 + CONV_LEN_2 + CONV_LEN_3 + CONV_LEN_4 + CONV_LEN_5 + CONV_LEN_6
# lstm
LSTM_SIZE1 = 256
LSTM_SIZE2 = 256
LSTM_DROP_RATE = 0.3
# dense
DENSE_SIZE1 = 512
DENSE_SIZE2 = 256

In [ ]:
def cnn_layer1(step_input, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    conv_output = conv(step_input)
    conv_output = GlobalMaxPool1D()(conv_output)
    return conv_output

def cnn_layer2(step_input, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    conv_output = conv(step_input)
    conv_output = GlobalAveragePooling1D()(conv_output)
    return conv_output

def cnn_layer3(step_input, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    conv_output = conv(step_input)
    conv_output1 = GlobalMaxPool1D()(conv_output)
    conv_output2 = GlobalAveragePooling1D()(conv_output)
    conv_output = concatenate([conv_output1, conv_output2])
    return conv_output

In [ ]:
from sklearn.model_selection import StratifiedKFold

best_results = []
last_results = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10, shuffle=True).split(X=word1, y=label)):  # word/char switch
    train_x1, train_x2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]  # word/char switch
    dev_x1, dev_x2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]  # word/char switch
    
    input1 = Input(shape=(SEQ_LEN,), dtype="int32")
    input2 = Input(shape=(SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],  # word/char switch
        output_dim=word_embedding_data.shape[1],  # word/char switch
        weights=[word_embedding_data],  # word/char switch
        input_length=SEQ_LEN,
        trainable=False
    )
    
    vector1 = embedding_layer(input1)
    vector2 = embedding_layer(input2)
    
    lstm_layer1 = Bidirectional(LSTM(LSTM_SIZE1, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True))
    layer1a = lstm_layer1(vector1)
    layer1b = lstm_layer1(vector2)
    lstm_layer2 = Bidirectional(LSTM(LSTM_SIZE2, dropout=DROP_RATE, recurrent_dropout=DROP_RATE, return_sequences=True))
    layer2a = lstm_layer2(layer1a)
    layer2b = lstm_layer2(layer1b)
    layer2a = concatenate([vector1, layer2a])
    layer2b = concatenate([vector2, layer2b])
    
    # TODO: 这里还可以添加BatchNorm层
    
    conv1a, conv1b = cnn_layer1(layer2a, filters=CONV_LEN_1, kernel_size=1), cnn_layer1(layer2b, filters=CONV_LEN_1, kernel_size=1)
    conv2a, conv2b = cnn_layer1(layer2a, filters=CONV_LEN_2, kernel_size=2), cnn_layer1(layer2b, filters=CONV_LEN_2, kernel_size=2)
    conv3a, conv3b = cnn_layer1(layer2a, filters=CONV_LEN_3, kernel_size=3), cnn_layer1(layer2b, filters=CONV_LEN_3, kernel_size=3)
    conv4a, conv4b = cnn_layer1(layer2a, filters=CONV_LEN_4, kernel_size=4), cnn_layer1(layer2b, filters=CONV_LEN_4, kernel_size=4)
    conv5a, conv5b = cnn_layer1(layer2a, filters=CONV_LEN_5, kernel_size=5), cnn_layer1(layer2b, filters=CONV_LEN_5, kernel_size=5)
    conv6a, conv6b = cnn_layer1(layer2a, filters=CONV_LEN_6, kernel_size=6), cnn_layer1(layer2b, filters=CONV_LEN_6, kernel_size=6)
    
    merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
    merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    diff = Lambda(lambda x: K.abs(x[0] - x[1]))([merge_a, merge_b])
    mult = Lambda(lambda x: x[0] * x[1])([merge_a, merge_b])
    merge = concatenate([diff, mult])
    
    x = Dropout(DROP_RATE)(merge)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE1, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE2, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[input1, input2], outputs=pred)
    model.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )

    early_stopping = EarlyStopping("val_loss", patience=PATIENCE)
    check_point = ModelCheckpoint(
        "./log/%s.TextRCNN.word.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
        monitor="val_loss",
        save_best_only=True,
    )
    
    fit_res = model.fit(
        x=[train_x1, train_x2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_x1, dev_x2], dev_y),
        shuffle=True,
        callbacks=[early_stopping, check_point]
    )
    
    pred_last = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    last_results.append(pd.DataFrame(pred_last, columns=["y_pre"]))
    
    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))
    pred_best = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    best_results.append(pd.DataFrame(pred_best, columns=["y_pre"]))

pd.DataFrame(pd.concat(last_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-TextRCNN_word_last.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)
pd.DataFrame(pd.concat(best_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-TextRCNN_word_best.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/30
228946/228946 [==============================] - 304s 1ms/step - loss: 0.3771 - acc: 0.8290 - val_loss: 0.2841 - val_acc: 0.8757
Epoch 2/30
228946/228946 [==============================] - 299s 1ms/step - loss: 0.2587 - acc: 0.8887 - val_loss: 0.2429 - val_acc: 0.8976
Epoch 3/30
228946/228946 [==============================] - 299s 1ms/step - loss: 0.2155 - acc: 0.9091 - val_loss: 0.2256 - val_acc: 0.9086
Epoch 4/30
228946/228946 [==============================] - 297s 1ms/step - loss: 0.1888 - acc: 0.9217 - val_loss: 0.2054 - val_acc: 0.9134
Epoch 5/30
228946/228946 [==============================] - 296s 1ms/step - loss: 0.1684 - acc: 0.9307 - val_loss: 0.2144 - val_acc: 0.9143
Epoch 6/30
228946/228946 [==============================] - 295s 1ms/step - loss: 0.1524 - acc: 0.9382 - val_loss: 0.2035 - val_acc: 0.9181
Epoch 7/30
228946/228946 [